In [1]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

In [7]:
# get data

boston = load_boston()
data = pd.DataFrame(data=boston.data, columns = boston.feature_names)
features = data.drop(['INDUS', 'AGE'], axis=1)

response = np.log(boston.target)
response = pd.DataFrame(response, columns=['PRICE'])
response.shape

/Users/alexmichel/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California 

(506, 1)

In [17]:
# using averages for columns and creating an array that holds the averages to use later in function
CRIME_IDX = 0
ZN_IDX = 1
CHAZ_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8


property_stats = features.mean().values.reshape(1,11)
property_stats

array([[3.61352356e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
        6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
        1.84555336e+01, 3.56674032e+02, 1.26530632e+01]])

In [16]:
features.mean().values.reshape(1,11)

array([[3.61352356e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
        6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
        1.84555336e+01, 3.56674032e+02, 1.26530632e+01]])

In [28]:
lin_reg = LinearRegression().fit(features, response)
fitted_vals = lin_reg.predict(features)

MSE = round(mean_squared_error(response, fitted_vals),3)
RMSE = round(np.sqrt(MSE),3)
print(f'The Mean Squared Error is {MSE}, and the Root Mean Squared Error is {RMSE}')

The Mean Squared Error is 0.035, and the Root Mean Squared Error is 0.187


In [61]:
# function that takes input and returns prediction, upper & lower bounds, and confidence interval
def get_log_estimate(num_rooms,
                    stud_per_classroom,
                    next_to_river=False,
                    high_confidence=True):
    
    #configure property
    property_stats[0][RM_IDX] = num_rooms
    property_stats[0][PTRATIO_IDX] = stud_per_classroom
    
    if next_to_river:
        property_stats[0][CHAZ_IDX] = 1
    else:
        property_stats[0][CHAZ_IDX] = 0
    
    #make prediction
    log_estimate = lin_reg.predict(property_stats)[0][0]
    
    # calc range
    if high_confidence:
        upper_bound = log_estimate + 2*RMSE
        lower_bound = log_estimate - 2*RMSE
        interval = 95
    else:
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 68
    
    
    
    return log_estimate, upper_bound, lower_bound, interval

In [60]:
get_log_estimate(3,20,1,True)

/Users/alexmichel/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


(2.7767581914803987, 3.150758191480399, 2.4027581914803986, 95)

In [62]:
np.median(boston.target)

21.2

Median House Price in Boston

799k

https://www.realtor.com/realestateandhomes-search/Boston_MA/overview

In [70]:
# scaling the boston house prices dataset to today's prices

REALTOR_MEDIAN_PRICE = 799.0
SCALE_FACTOR = REALTOR_MEDIAN_PRICE / np.median(boston.target)
SCALE_FACTOR

log_est, upper, lower, conf = get_log_estimate(9, 15, False, False)

# convert to today's dollars

dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
dollar_low = np.e**lower * 1000 * SCALE_FACTOR

# round to nearest thousand

rounded_est = np.around(dollar_est, -3)
rounded_hi = np.around(dollar_hi, -3)
rounded_low = np.around(dollar_low, -3)

print(f'The estimated property value is {rounded_est}.')
print(f'At {conf}% confidence the valuation range is')
print(f'USD {rounded_low} at the lower end to USD {rounded_hi} at the high end.')

The estimated property value is 1132000.0.
At 68% confidence the valuation range is
USD 939000.0 at the lower end to USD 1365000.0 at the high end.


/Users/alexmichel/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [92]:
# new function to do conversions
def dollar_estimate(rm, ptratio, chas=False, large_range=True):
    """
    Function to take in the get_log_estimate, do conversions, and return prediction with upper/lower/intervals
    Keyword arguements:
    rm -- number of rooms
    ptratio -- number of students per teacher in the classroom for the school in the area
    chas -- boolean, on the river or not
    large_range -- boolean, 68 or 95 confidence interval
    """
    if rm < 1 or ptratio < 1:
        print('Impossible. Try again.')
        return
    
    log_est, upper, lower, conf = get_log_estimate(num_rooms=rm, stud_per_classroom=ptratio, next_to_river=chas, high_confidence=large_range)
    REALTOR_MEDIAN_PRICE = 799.0
    SCALE_FACTOR = REALTOR_MEDIAN_PRICE / np.median(boston.target)
    # convert to today's dollars

    dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
    dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
    dollar_low = np.e**lower * 1000 * SCALE_FACTOR

    # round to nearest thousand

    rounded_est = np.around(dollar_est, -3)
    rounded_hi = np.around(dollar_hi, -3)
    rounded_low = np.around(dollar_low, -3)

    print(f'The estimated property value is {rounded_est}.')
    print(f'At {conf}% confidence the valuation range is')
    print(f'USD {rounded_low} at the lower end to USD {rounded_hi} at the high end.')

In [91]:
dollar_estimate(rm=6, ptratio=12, chas=True)

The estimated property value is 1072000.0.
At 95% confidence the valuation range is
USD 738000.0 at the lower end to USD 1559000.0 at the high end.


/Users/alexmichel/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [93]:
import boston_valuation as val
val.dollar_estimate(rm=6, ptratio=12, chas=True)

The estimated property value is 1072000.0.
At 95% confidence the valuation range is
USD 738000.0 at the lower end to USD 1559000.0 at the high end.


/Users/alexmichel/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California 